Individual Assignment 2 Task 1
---
---

Name: Ryan Hong Yang Tan

UOW ID: 8560341

Reading the csv file into a dataframe and checking for its length

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import CategoricalNB, GaussianNB
from sklearn.metrics import accuracy_score, classification_report

from google.colab import drive
drive.mount("/content/drive")

# Reading the csv file
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/secondary_data.csv', sep=';')
# Checking if the data size is correct
print(len(data))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
61069 



**Data pre-processing**

---

Checking for null values

In [ ]:
# Checking for null values
print('Null values present before pre processing:')
print(data.isnull().sum())

Null values present before pre processing:
class                       0
cap-diameter                0
cap-shape                   0
cap-surface             14120
cap-color                   0
does-bruise-or-bleed        0
gill-attachment          9884
gill-spacing            25063
gill-color                  0
stem-height                 0
stem-width                  0
stem-root               51538
stem-surface            38124
stem-color                  0
veil-type               57892
veil-color              53656
has-ring                    0
ring-type                2471
spore-print-color       54715
habitat                     0
season                      0
dtype: int64


Dataset only has 61,069 rows, the following columns has more than half of it possessing null values and as such I have decided to remove them

In [ ]:
data.drop('stem-root', axis='columns', inplace=True)
data.drop('veil-type', axis='columns', inplace=True)
data.drop('veil-color', axis='columns', inplace=True)
data.drop('spore-print-color', axis='columns', inplace=True)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61069 entries, 0 to 61068
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   class                 61069 non-null  object 
 1   cap-diameter          61069 non-null  float64
 2   cap-shape             61069 non-null  object 
 3   cap-surface           46949 non-null  object 
 4   cap-color             61069 non-null  object 
 5   does-bruise-or-bleed  61069 non-null  object 
 6   gill-attachment       51185 non-null  object 
 7   gill-spacing          36006 non-null  object 
 8   gill-color            61069 non-null  object 
 9   stem-height           61069 non-null  float64
 10  stem-width            61069 non-null  float64
 11  stem-surface          22945 non-null  object 
 12  stem-color            61069 non-null  object 
 13  has-ring              61069 non-null  object 
 14  ring-type             58598 non-null  object 
 15  habitat            

Categorical values are cleaned by filling with the mode while continuous values are cleaned by filling with the mean

In [ ]:
# Pre-processing
def preprocessing(dataName):
  for col in data.columns:
    # Categorical data are cleaned using mode
    if dataName[col].dtype == 'object':
      # Fills null data with the mode by selecting the mode value of the column
      dataName[col] = dataName[col].fillna(dataName[col].mode()[0])
    # Continuous data are cleaned using mean
    else:
      # Fills null data with the mean value
      dataName[col] = dataName[col].fillna(dataName[col].mean())
  return dataName

# Running the pre processing function
data = preprocessing(data)

# Checking for null values
print('\nNull values present after pre processing:')
print(data.isnull().sum())


Null values present after pre processing:
class                   0
cap-diameter            0
cap-shape               0
cap-surface             0
cap-color               0
does-bruise-or-bleed    0
gill-attachment         0
gill-spacing            0
gill-color              0
stem-height             0
stem-width              0
stem-surface            0
stem-color              0
has-ring                0
ring-type               0
habitat                 0
season                  0
dtype: int64


Checking for datatype

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61069 entries, 0 to 61068
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   class                 61069 non-null  object 
 1   cap-diameter          61069 non-null  float64
 2   cap-shape             61069 non-null  object 
 3   cap-surface           61069 non-null  object 
 4   cap-color             61069 non-null  object 
 5   does-bruise-or-bleed  61069 non-null  object 
 6   gill-attachment       61069 non-null  object 
 7   gill-spacing          61069 non-null  object 
 8   gill-color            61069 non-null  object 
 9   stem-height           61069 non-null  float64
 10  stem-width            61069 non-null  float64
 11  stem-surface          61069 non-null  object 
 12  stem-color            61069 non-null  object 
 13  has-ring              61069 non-null  object 
 14  ring-type             61069 non-null  object 
 15  habitat            

Data encoding

In [ ]:
encoder = LabelEncoder()
for col in data.select_dtypes(include=['object']).columns:
  data[col] = encoder.fit_transform(data[col])

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61069 entries, 0 to 61068
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   class                 61069 non-null  int64  
 1   cap-diameter          61069 non-null  float64
 2   cap-shape             61069 non-null  int64  
 3   cap-surface           61069 non-null  int64  
 4   cap-color             61069 non-null  int64  
 5   does-bruise-or-bleed  61069 non-null  int64  
 6   gill-attachment       61069 non-null  int64  
 7   gill-spacing          61069 non-null  int64  
 8   gill-color            61069 non-null  int64  
 9   stem-height           61069 non-null  float64
 10  stem-width            61069 non-null  float64
 11  stem-surface          61069 non-null  int64  
 12  stem-color            61069 non-null  int64  
 13  has-ring              61069 non-null  int64  
 14  ring-type             61069 non-null  int64  
 15  habitat            

Separating dataset into categorical and continuous datatypes and into X and Y

In [ ]:
categorical_features = [col for col in data.columns if data[col].dtype == 'int64' and col != 'class']
continuous_features = [col for col in data.columns if data[col].dtype == 'float64' and col != 'class']

x_cat = data[categorical_features]
x_cont = data[continuous_features]
y = data['class']

x_cat.info()
x_cont.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61069 entries, 0 to 61068
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   cap-shape             61069 non-null  int64
 1   cap-surface           61069 non-null  int64
 2   cap-color             61069 non-null  int64
 3   does-bruise-or-bleed  61069 non-null  int64
 4   gill-attachment       61069 non-null  int64
 5   gill-spacing          61069 non-null  int64
 6   gill-color            61069 non-null  int64
 7   stem-surface          61069 non-null  int64
 8   stem-color            61069 non-null  int64
 9   has-ring              61069 non-null  int64
 10  ring-type             61069 non-null  int64
 11  habitat               61069 non-null  int64
 12  season                61069 non-null  int64
dtypes: int64(13)
memory usage: 6.1 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61069 entries, 0 to 61068
Data columns (total 3 columns):
 #   Colu

Train test split, 80% train 20% split

In [ ]:
x_cat_train, x_cat_test, x_cont_train, x_cont_test, y_train, y_test = train_test_split(x_cat, x_cont, y, test_size=0.2, random_state=42, stratify=y)

print(x_cat_train.shape)
print(x_cat_test.shape)
print(x_cont_train.shape)
print(x_cont_test.shape)
print(y_train.shape)
print(y_test.shape)

(48855, 13)
(12214, 13)
(48855, 3)
(12214, 3)
(48855,)
(12214,)


Training and Evaluation using CategoricalNB (categorical feature)

In [ ]:
categoricalNB = CategoricalNB()
categoricalNB.fit(x_cat_train, y_train)
y_pred_cat = categoricalNB.predict(x_cat_test)

print(f"CategoricalNB Accuracy: {accuracy_score(y_test, y_pred_cat):.4f}")
print("CategoricalNB Classification Report:\n", classification_report(y_test, y_pred_cat))

CategoricalNB Accuracy: 0.7556
CategoricalNB Classification Report:
               precision    recall  f1-score   support

           0       0.72      0.74      0.73      5436
           1       0.79      0.77      0.78      6778

    accuracy                           0.76     12214
   macro avg       0.75      0.75      0.75     12214
weighted avg       0.76      0.76      0.76     12214



Training and Evaluation using GaussianNB (continuous feature)

In [ ]:
scaler = StandardScaler()
x_cont_train_scaled = scaler.fit_transform(x_cont_train)
x_cont_test_scaled = scaler.fit_transform(x_cont_test)

gaussianNB = GaussianNB()
gaussianNB.fit(x_cont_train_scaled, y_train)
y_pred_gauss = gaussianNB.predict(x_cont_test_scaled)

print(f"GaussianNB Accuracy: {accuracy_score(y_test, y_pred_gauss):.4f}")
print("GaussianNB Classification Report:\n", classification_report(y_test, y_pred_gauss))

GaussianNB Accuracy: 0.5699
GaussianNB Classification Report:
               precision    recall  f1-score   support

           0       0.54      0.25      0.34      5436
           1       0.58      0.83      0.68      6778

    accuracy                           0.57     12214
   macro avg       0.56      0.54      0.51     12214
weighted avg       0.56      0.57      0.53     12214



In [ ]:
# Get probability estimates from both models
probs_cat = categoricalNB.predict_proba(x_cat_test)  # CategoricalNB probabilities
probs_gauss = gaussianNB.predict_proba(x_cont_test_scaled)  # GaussianNB probabilities

# Combine probabilities by multiplying them element-wise
final_probs = probs_cat * probs_gauss

# Predict final class using argmax
y_pred_hybrid = np.argmax(final_probs, axis=1)

# Evaluate the hybrid model
print(f"Hybrid Naïve Bayes Accuracy: {accuracy_score(y_test, y_pred_hybrid):.4f}")
print("Hybrid Naïve Bayes Classification Report:\n", classification_report(y_test, y_pred_hybrid))

Hybrid Naïve Bayes Accuracy: 0.6868
Hybrid Naïve Bayes Classification Report:
               precision    recall  f1-score   support

           0       0.71      0.50      0.59      5436
           1       0.68      0.83      0.75      6778

    accuracy                           0.69     12214
   macro avg       0.69      0.67      0.67     12214
weighted avg       0.69      0.69      0.68     12214



In this task, i used both CategoricalNB and GaussianNB as they each handle specific datatypes. The results i got shows that the CategoricalNB performed better with a accuracy of 0.76 compared to GaussianNB accuracy of 0.57. However, this could be due to lack of continuous feature as there are only 3 continuous feature present resulting in low feature complexity and as such the difference in accuracy.


In order to leverage on both CategoricalNB and GaussianNB, I experimented with a hybrid model combining the probability of estimation from both models and using argmax to make a final prediction as it could potentially improve performance.